## 必要なライブラリをインポート

In [6]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

In [ ]:
# time.sleepを使ってサーバーへの負担を軽減
START_URL = "https://www.musashino-u.ac.jp/"
ROOT_DOMAIN = urlparse(START_URL).netloc
WAIT_SEC = 1.2
# 画像や動画などをスキップする拡張子リスト
SKIP_EXTS = [
    '.pdf', '.png', '.jpg', '.jpeg', '.gif', '.svg',
    '.mp4', '.mp3', '.zip', '.ico', '.css', '.js', '.webp'
]

# urlを管理し辞書に格納
queue_urls = [START_URL]
checked_urls = set()
site_titles = {}

print("🚀 クロール開始！\n")

page_num = 0

# htmlパース、タグオブジェクト探索と抽出(授業資料参考)
while queue_urls:
    target = queue_urls.pop(0)

    if target in checked_urls:
        continue

    checked_urls.add(target)
    time.sleep(WAIT_SEC)

    print(f"▶ {target}")

    try:
        # Webサーバにクエリを発行
        resp = requests.get(
            target,
            headers={"User-Agent": "MU-WebMapper/1.0"},
            timeout=8
        )

        if resp.status_code != 200:
            print(f"  ❌ ステータスコード {resp.status_code} (スキップ)")
            continue
        # htmlをパースしてデータを取得(授業資料参考)
        resp.encoding = resp.apparent_encoding
        html = BeautifulSoup(resp.text, "html.parser")

        # ---- タイトル取得 ----
        t = html.title.string.strip() if html.title and html.title.string else "タイトルなし"
        site_titles[target] = t

        page_num += 1
        print(f"  ✅ [{page_num:03}] {t}")

        # html.find_all('a', href=True) によるリンク探索(授業資料参考)
        for a in html.find_all("a", href=True):
            link = urljoin(target, a["href"])
            parsed = urlparse(link)
            normalized = parsed._replace(query="", fragment="").geturl()

            if (
                parsed.netloc == ROOT_DOMAIN
                and normalized not in checked_urls
                and normalized not in queue_urls
                and not any(normalized.lower().endswith(ext) for ext in SKIP_EXTS)
            ):
                queue_urls.append(normalized)

    except Exception as err:
        print(f"  ⚠ エラー発生: {err}")
        continue

# 辞書型変数をプリントで表示
print("\n🌐 --- クロール完了 ---")
for link, title in site_titles.items():
    print(f"・{title}: {link}")

print(f"\n合計 {len(site_titles)} ページを検出しました。")


🚀 クロール開始！

▶ https://www.musashino-u.ac.jp/
  ✅ [001] 武蔵野大学
▶ https://www.musashino-u.ac.jp/access.html
  ✅ [002] 交通アクセス | 武蔵野大学
▶ https://www.musashino-u.ac.jp/admission/request.html
  ✅ [003] 資料請求 | 入試情報 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/contact.html
  ✅ [004] お問い合わせ | 武蔵野大学
▶ https://www.musashino-u.ac.jp/prospective-students.html
  ✅ [005] 武蔵野大学で学びたい方 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/students.html
  ✅ [006] 在学生の方 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/alumni.html
  ✅ [007] 卒業生の方 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/parents.html
  ✅ [008] 保護者の方 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/business.html
  ✅ [009] 企業・研究者の方 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/guide/
  ✅ [010] 大学案内 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/guide/profile/
  ✅ [011] 大学紹介 | 大学案内 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/guide/activities/
  ✅ [012] 大学の取り組み | 大学案内 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/guide/campus/
  ✅ [013] キャンパス | 大学案内 | 武蔵野大学
▶ https://www.musashino-u.ac.jp/guide/facility/
 